In [2]:
import os
import fnmatch
import cv2
import random
from datetime import datetime
import numpy as np
from sklearn.utils import shuffle

In [3]:
# # Load the TensorBoard notebook extension
# %load_ext tensorboard

In [4]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

# Load images as numpy array

In [5]:
images = {"radial": {}, "non-radial": {}}
for path, sub_dirs, files in os.walk("data/cropped/16x16"):
    for file in fnmatch.filter(files, "*.png"):
        file_path = os.path.join(path, file)
        img = file_path.split("/")[-1].split("_")[0]
        if "non-radial" in file_path:
            images['non-radial'].update({img: images['non-radial'].get(img, []) + [cv2.imread(file_path)]})
        else:
            images['radial'].update({img: images['radial'].get(img, []) + [cv2.imread(file_path)]})

### Shape of image

In [12]:
in_shape = images['radial'][list(images['radial'].keys())[0]][0].shape
in_shape

(120, 120, 3)

# Hyperparameters

In [6]:
batch_size = 128
epochs = 10
verbose = 1
learning_rate = 0.001
# mom = 0.9 # momentum
input_shape = in_shape # (120, 120, 3) if 16x16 images selected

# Create cross validation set - leave one out

In [7]:
random.seed(1)
non_radial_rand = random.sample([i for i in images['non-radial'].keys() if i not in list(images['radial'].keys())], k = images['radial'].__len__())
dist = []
for idx, i in enumerate(list(images['radial'].keys())):
    choices = [i, non_radial_rand[idx]]
    rem = list(set(i for k, v in images.items() for i in v.keys() if i not in choices))
    dist.append([rem, choices])

### Now we have seven folds where in each fold, one image from radial and one from non-radial are left out for model evaluation

# Define the model architecture

In [8]:
cnn_model = Sequential([
    Conv2D(8, (3,3), activation='relu', input_shape=in_shape, name="CONVOLUTION_2D_1"),
    MaxPooling2D((2,2), name="MAXPOOL_2D_1"),
    Conv2D(16, (3,3), activation='relu', name='CONVOLUTION_2D_2'),
    MaxPooling2D((2,2), name="MAXPOOL_2D_2"),
    Conv2D(32, (3,3), activation='relu', name='CONVOLUTION_2D_3'),
    MaxPooling2D((2,2), name="MAXPOOL_2D_3"),
    Flatten(name="FLATTEN"),
    Dense(256, activation='relu', name="DENSE"),
    Dense(1, activation='sigmoid', name="SIGMOID")
], name='RADIAL_CNN')

# Compile the model

In [9]:
cnn_model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=learning_rate),
    metrics=['accuracy']
)
cnn_model.summary()

Model: "RADIAL_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
CONVOLUTION_2D_1 (Conv2D)    (None, 118, 118, 8)       224       
_________________________________________________________________
MAXPOOL_2D_1 (MaxPooling2D)  (None, 59, 59, 8)         0         
_________________________________________________________________
CONVOLUTION_2D_2 (Conv2D)    (None, 57, 57, 16)        1168      
_________________________________________________________________
MAXPOOL_2D_2 (MaxPooling2D)  (None, 28, 28, 16)        0         
_________________________________________________________________
CONVOLUTION_2D_3 (Conv2D)    (None, 26, 26, 32)        4640      
_________________________________________________________________
MAXPOOL_2D_3 (MaxPooling2D)  (None, 13, 13, 32)        0         
_________________________________________________________________
FLATTEN (Flatten)            (None, 5408)              0

# Define the Keras TensorBoard callback

In [10]:
# logdir="logs/cnn_model/" + datetime.now().strftime("%y_%m_%d-%H_%M_%S_%s")
# tensorboard_callback = TensorBoard(log_dir=logdir)

In [11]:
acc_per_fold = []
loss_per_fold = []
history = []
fold_no = 1

for i in dist:
    inputs, outputs = {"train": [], "test": []}, {"train": [], "test": []}
    for keys, vals in images.items():
        bool_val = 0 if keys == "radial" else 1
        for k, v in vals.items():
            var = 'train' if k in i[0] else 'test'
            inputs[var].extend(v)
            outputs[var].extend([bool_val] * len(v))
    for j in [inputs, outputs]:
        for k in j.keys():
            j[k] = np.stack(j[k], axis=0)
    inputs['train'], outputs['train'] = shuffle(inputs['train'], outputs['train'])
    inputs['test'], outputs['test'] = shuffle(inputs['test'], outputs['test'])
    inputs['train'], inputs['test'] = inputs['train']/255., inputs['test']/255.
    
    # Generate a print
    print('-----------------------------------------------------------------')
    print('Training for fold', fold_no, '...')

    # Fit data to model
    history.append(cnn_model.fit(inputs['train'], outputs['train'],
                                 batch_size=batch_size,
                                 epochs=epochs,
                                 verbose=verbose,
#                                  callbacks=[TensorBoard(log_dir=logdir+str(fold_no))],
                                 validation_data=(inputs['test'], outputs['test'])
                                )
                  )
#     Generate generalization metrics
    scores = cnn_model.evaluate(inputs['test'], outputs['test'], verbose=0)
    print('Score for fold {0}: {1} of {2}; {3} of {4}%'.format(fold_no, cnn_model.metrics_names[0], scores[0], cnn_model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no += 1
    
# == Provide average scores ==
print('=================================================================')
print('Score per fold')
for idx, i in enumerate(zip(loss_per_fold, acc_per_fold)):
    print('-----------------------------------------------------------------')
    print('> Fold {0} - Loss: {1:.6f} - Accuracy: {2:.6f}'.format(idx+1, i[0], i[1]))
print('=================================================================')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('=================================================================')

-----------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
19/19 [==============================] - 2s 113ms/step - loss: 0.5670 - accuracy: 0.7826 - val_loss: 0.4692 - val_accuracy: 0.8305
Epoch 2/10
19/19 [==============================] - 2s 80ms/step - loss: 0.5086 - accuracy: 0.7920 - val_loss: 0.4684 - val_accuracy: 0.8305
Epoch 3/10
19/19 [==============================] - 2s 81ms/step - loss: 0.5018 - accuracy: 0.7972 - val_loss: 0.4590 - val_accuracy: 0.8305
Epoch 4/10
19/19 [==============================] - 1s 79ms/step - loss: 0.4763 - accuracy: 0.8079 - val_loss: 0.5108 - val_accuracy: 0.8305
Epoch 5/10
19/19 [==============================] - 1s 77ms/step - loss: 0.4875 - accuracy: 0.8160 - val_loss: 0.4522 - val_accuracy: 0.8305
Epoch 6/10
19/19 [==============================] - 1s 77ms/step - loss: 0.4480 - accuracy: 0.8169 - val_loss: 0.4500 - val_accuracy: 0.8305
Epoch 7/10
19/19 [==============================] - 1s 79ms/ste

In [12]:
# %tensorboard --logdir logs